In [ ]:
import glob
import importlib
import os

In [ ]:
import cv2
import numpy as np
import pandas as pd
import scipy

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import nitelite_mapmaker

In [ ]:
importlib.reload(nitelite_mapmaker)

# Settings

In [ ]:
flight_name = '220513-FH135'
data_dir = '/Users/Shared/data'
google_drive_dir = '/Users/zhafensaavedra/Google Drive'

In [ ]:
image_dir = os.path.join(data_dir, 'nitelite/images', flight_name, '23085686' )
metadata_dir = os.path.join(google_drive_dir, 'Shared drives/NITELite/Data & Analysis/Old NITELite Flights', flight_name, 'data')
combined_externally_metadata_fp = os.path.join(metadata_dir, 'CollatedImageLog.csv')
metadata_fp = os.path.join(metadata_dir, 'image.log')
imu_data_fp = os.path.join(metadata_dir, 'OBC/PresIMULog.csv')
gps_data_fp = os.path.join(metadata_dir, 'OBC/GPSLog.csv')

In [ ]:
percent_for_landed = 95.
percent_for_cruising = 10.
mult_of_std_for_steady = 2.
column_for_steady = 'imuGyroX'
rolling_window_in_min = 1.

# offset between image and UTC based on timezone
img_to_metadata_tz_offset_in_hr = 5

In [ ]:
# Ground altitude in meters
ground_alt = 220.

In [ ]:
decent_test_seeds = [111, 1631489, ]
rng = np.random.default_rng(decent_test_seeds[0])

## Setup

In [ ]:
mm = nitelite_mapmaker.mapmaker.Mapmaker(image_dir=image_dir, metadata_fp=metadata_fp)

# Metadata

## Image Log

In [ ]:
metadata = pd.read_csv(
    metadata_fp,
    names=['odroid_timestamp', 'obc_timestamp', 'camera_num', 'serial_num', 'exposure_time', 'sequence_ind', 'internal_temp', 'filename', ] + ['Unnamed: {}'.format(i+1) for i in range(12)]
)

In [ ]:
# Parse the timestamp
# We use a combination of the odroid timestamp and the obc timestamp because
# the odroid timestamp is missing the year but the obc_timestamp has the wrong month.
timestamp_split = metadata['obc_timestamp'].str.split('_')
metadata['obc_timestamp'] = pd.to_datetime(timestamp_split.apply(lambda x: '_'.join(x[:2])), format=' %Y%m%d_%H%M%S' )
metadata['timestamp'] = pd.to_datetime(metadata['obc_timestamp'].dt.year.astype(str) + ' ' + metadata['odroid_timestamp'])
metadata['timestamp_id'] = timestamp_split.apply(lambda x: x[-1]).astype(int)

In [ ]:
# Drop unnamed columns
metadata = metadata.drop([ column for column in metadata.columns if 'Unnamed' in column ], axis='columns')

## IMU Data

### Clean data

In [ ]:
imu_data = pd.read_csv(imu_data_fp, low_memory=False)

In [ ]:
# Remove the extra header rows, and the nan rows
imu_data.dropna(subset=['CurrTimestamp',], inplace=True)
imu_data.drop(imu_data.index[imu_data['CurrTimestamp'] == 'CurrTimestamp'], inplace=True)

In [ ]:
# Handle some situations where the pressure is negative
ac_columns = ['TempC', 'pressure', 'mAltitude']
imu_data.loc[imu_data['pressure'].astype(float) < 0,] = np.nan

In [ ]:
# Convert to datetime, toss out IMU recordings not associated with the 5-13 flight.
imu_data['CurrTimestamp'] = pd.to_datetime(imu_data['CurrTimestamp'])
imu_data.drop(imu_data.index[imu_data['CurrTimestamp'] < pd.to_datetime('2022-5-13 20')], inplace=True)

In [ ]:
# Sort by datetime
imu_data.sort_values('CurrTimestamp',inplace=True)

In [ ]:
# Assign dtypes
skipped_cols = []
for column in imu_data.columns:
    if column == 'CurrTimestamp':
        continue
        
    imu_data[column] = imu_data[column].astype(float)

In [ ]:
# Now also handle when the altitude is weird or the temperature is weird
imu_data.loc[imu_data['TempC'] < -273,ac_columns] = np.nan
imu_data.loc[imu_data['mAltitude'] < 0.,ac_columns] = np.nan
imu_data.loc[imu_data['mAltitude'] > 20000.,ac_columns] = np.nan

### Parse into flight phases

We can use the pressure to divide the flight up into approximate phases.

In [ ]:
# We'll divide up into phases based on pressure relative to max or min pressure.
p_max = imu_data['pressure'].max()
p_min = imu_data['pressure'].min()
p_diff = p_max - p_min
p_landed = p_min + percent_for_landed / 100. * p_diff
p_cruising = p_min + percent_for_cruising / 100. * p_diff

In [ ]:
# Identify the transition phases
phases = ['pre-flight', 'ascent', 'cruise', 'descent', 'post-flight'] # For reference
phase_values = []
transition_indices = []
j = 0
for i, p in enumerate(imu_data['pressure']):
    
    # Pre-flight to ascent
    if j == 0:
        if p < p_landed:
            transition_indices.append(imu_data.index[i])
            j += 1
    elif j == 1:
        if p < p_cruising:
            transition_indices.append(imu_data.index[i])
            j += 1
    elif j == 2:
        if p > p_cruising:
            transition_indices.append(imu_data.index[i])
            j += 1
    elif j == 3:
        if p > p_landed:
            transition_indices.append(imu_data.index[i])
            j += 1
    else:
        pass

    phase_values.append(j)
imu_data['flight_phase_num'] = phase_values
imu_data['flight_phase'] = np.array(phases)[imu_data['flight_phase_num']]

In [ ]:
# View versus time
g = sns.PairGrid(imu_data, x_vars=['CurrTimestamp'], aspect=3, hue='flight_phase')
g.map_offdiag(sns.histplot)

for ax_row in g.axes:
    ax = ax_row[0]
    
    if ax.get_ylabel() == 'pressure':
        ax.axhline(
            p_landed,
            color = 'k',
            linestyle = '--',
        )
        ax.axhline(
            p_cruising,
            color = 'k',
            linestyle = '--',
        )
    
    for ind in transition_indices:
        ax.axvline(
            imu_data.loc[ind,'CurrTimestamp'],
            color = 'k',
            linestyle = '--',
        )
        
min_times = imu_data.groupby('flight_phase')['CurrTimestamp'].min()
for phase in min_times.index:
    ax.annotate(
        text = phase,
        xy = (min_times.loc[phase], 1),
        xycoords = matplotlib.transforms.blended_transform_factory(ax.transData, ax.transAxes),
        # xytext = (5,5),
        # textcoords = 'offset points',
    )

### Select the steady cruise regime

In addition to parsing the flight into phases, we can identify the part of the flight where the cruise is steady.

In [ ]:
# Select cruise data
cruise_data = imu_data.loc[imu_data['flight_phase'] == 'cruise']
cruise_data = cruise_data.set_index('CurrTimestamp')

In [ ]:
# Get rolling deviation
cruise_rolling = cruise_data.rolling(window=pd.Timedelta(rolling_window_in_min, 'min'))
cruise_rolling_std = cruise_rolling.std(numeric_only=True)

In [ ]:
# Identify and store steady data
cruise_data.loc[:,'is_steady'] = cruise_rolling_std[column_for_steady] < mult_of_std_for_steady * np.nanmedian(cruise_rolling_std[column_for_steady])
cruise_rolling_std.loc[:,'is_steady'] = cruise_data['is_steady']
imu_data['is_steady'] = False
imu_data.loc[imu_data['flight_phase'] == 'cruise','is_steady'] = cruise_data['is_steady'].values

In [ ]:
# View versus time
g = sns.PairGrid(cruise_rolling_std.reset_index(), x_vars=['CurrTimestamp'], hue='is_steady', aspect=3)
g.map_offdiag(sns.histplot)

for ax_row in g.axes:
    ax = ax_row[0]
    y_key = ax.get_ylabel()
    med_std = np.nanmedian(cruise_rolling_std[y_key])
    ax.axhline(
        mult_of_std_for_steady * med_std,
        color = 'k',
        linestyle = '--',
    )

### Steady flight visual inspection

This is the cleanest data we could hope for, so let's take a look at it.

In [ ]:
steady_data = imu_data.loc[imu_data['is_steady']]

In [ ]:
instruments = {
    'Accel': [ 'imuAccelX', 'imuAccelY', 'imuAccelZ', ],
    'Gyro': [ 'imuGyroX', 'imuGyroY', 'imuGyroZ', ],
    'Mag': [ 'imuMagX', 'imuMagY', 'imuMagZ', ],
    'RollPitchYaw': [ 'imuRoll', 'imuPitch', 'imuYaw', ],
}

In [ ]:
mosaic = [ [ _, ] for _ in instruments.keys() ]

fig = plt.figure(figsize=(20,10))
ax_dict = fig.subplot_mosaic(mosaic)

for inst_name, inst_keys in instruments.items():
    
    ax = ax_dict[inst_name]
    
    for inst_key in inst_keys:
        sns.scatterplot(
            steady_data,
            x = 'CurrTimestamp',
            y = inst_key,
            ax = ax,
            edgecolor=None,
            s = 10,
            label = inst_key,
        )
        
    ax.legend()
    ax.set_ylabel(inst_name)

## GPS Data

### Clean data

In [ ]:
gps_data = pd.read_csv(gps_data_fp)

In [ ]:
# Remove the extra header rows and the empty rows
gps_data.dropna(subset=['CurrTimestamp', ], inplace=True)
gps_data.drop(gps_data.index[gps_data['CurrTimestamp'] == 'CurrTimestamp'], inplace=True)

In [ ]:
# Remove the empty rows
gps_data.drop(gps_data.index[gps_data['CurrTimestamp'] == '00.00.0000 00:00:00000'], inplace=True)

In [ ]:
# Convert to datetime, toss out recordings not associated with the flight itself.
gps_data['CurrTimestamp'] = pd.to_datetime(gps_data['CurrTimestamp'])
gps_data.drop(gps_data.index[gps_data['CurrTimestamp'] < pd.to_datetime('2022-5-13 20')], inplace=True)

In [ ]:
# Assign dtypes
for column in gps_data.columns:
    if column in ['CurrTimestamp', 'GPSTime']:
        continue
        
    gps_data[column] = gps_data[column].astype(float)

### Compare to IMU Data

In [ ]:
# Look at acceleration data to identify launch point
accel_data = imu_data[['CurrTimestamp', 'imuAccelX', 'imuAccelY', 'imuAccelZ']]
accel_data.set_index('CurrTimestamp', inplace=True)
accel_data = accel_data.dropna()

In [ ]:
# Assume the maximum rolling acceleration during the pre-flight marks the launch
accel_rolling_mean = accel_data.rolling(window=pd.Timedelta(rolling_window_in_min*0.5, 'min')).mean()
rolling_mean_mag = np.linalg.norm(accel_rolling_mean, axis=1)
end_of_preflight = imu_data.loc[imu_data['flight_phase']=='pre-flight', 'CurrTimestamp'].max()
launch_time = accel_data.index[np.argmax(rolling_mean_mag[accel_data.index<end_of_preflight])]

In [ ]:
fig = plt.figure(figsize=(20,10))
ax_dict = fig.subplot_mosaic([['overall', 'zoomed', ]])

xlims = {
    'overall': None,
    'zoomed': (launch_time - pd.Timedelta(1, 'min'), launch_time + pd.Timedelta(1, 'min')),
}
for ax_key in ['overall', 'zoomed']:
    
    ax = ax_dict[ax_key]
    
    if xlims[ax_key] is not None:
        gps_selected = gps_data.loc[(
            (gps_data['CurrTimestamp'] > xlims[ax_key][0]) &
            (gps_data['CurrTimestamp'] < xlims[ax_key][1])
        )]
        imu_selected = imu_data.loc[(
            (imu_data['CurrTimestamp'] > xlims[ax_key][0]) &
            (imu_data['CurrTimestamp'] < xlims[ax_key][1])
        )]
    else:
        gps_selected = gps_data
        imu_selected = imu_data
    
    sns.scatterplot(
        gps_selected,
        x = 'CurrTimestamp',
        y = 'GPSAlt',
        edgecolor = None,
        label = 'GPS',
        ax = ax,
        zorder = 100,
    )
    sns.scatterplot(
        imu_selected,
        x = 'CurrTimestamp',
        y = 'mAltitude',
        edgecolor = None,
        label = 'IMU',
        ax = ax,
    )
    
    ax.axvline(
        launch_time,
        color = '0.5',
        linewidth = 1,
    )
    
    if ax_key == 'overall':
        ax2 = ax.twinx()
        ax2.plot(
            accel_data.index,
            rolling_mean_mag,
            color = 'k',
        )
        ax2.set_ylabel('rolling acceleration')
    ax.legend()
    
# ax.scatter(
#     gps_data['CurrT
# )

The match-up between the GPS and IMU data is not bad. We'll try using them as-is. The magnitude of the acceleration matches with the launch time to within a minute, but is still off by ~30 seconds.

## Combine

In [ ]:
dfs_interped = [metadata,]
# img_timestamps = metadata.index.get_level_values(1)
source_names = ['imu', 'gps']
for i, df_to_include in enumerate([imu_data, gps_data]):
    
    df_to_include = df_to_include.copy()

    # Polish up for interpolation
    if i == 0:
        del df_to_include['flight_phase']
        df_to_include['is_steady'] = df_to_include['is_steady'].astype(int)
    elif i == 1:
        del df_to_include['GPSTime']

    # Get the timestamps in the right time zone
    df_to_include['timestamp_img_tz'] = df_to_include['CurrTimestamp'] - pd.Timedelta(img_to_metadata_tz_offset_in_hr, 'hr')
    df_to_include = df_to_include.dropna(subset=['timestamp_img_tz']).set_index('timestamp_img_tz').sort_index()
    df_to_include['timestamp_int_{}'.format(source_names[i])] = df_to_include['CurrTimestamp'].astype(int)
    del df_to_include['CurrTimestamp']

    # Interpolate
    interp_fn = scipy.interpolate.interp1d(df_to_include.index.astype(int), df_to_include.values.transpose())
    interped = interp_fn(metadata['timestamp'].astype(int))
    df_interped = pd.DataFrame(interped.transpose(), columns=df_to_include.columns)
    
    dfs_interped.append(df_interped)

In [ ]:
metadata = pd.concat(dfs_interped, axis='columns', )

In [ ]:
# Set up a useful index
metadata['id'] = metadata.index
metadata = metadata.set_index(['camera_num', 'timestamp', 'id']).sort_index()

In [ ]:
metadata

# Image Calibration

## Inspect a Random Image

In [ ]:
fp = rng.choice(mm.flight.image_fps)

In [ ]:
img = mm.flight.get_rgb_img(fp)

In [ ]:
mm.data_viewer.plot_img(img)

In [ ]:
bins = np.arange(-0.5, mm.flight.max_val-0.5, 1)
plt_types = ['big_picture', 'zoomed', 'zoomed_upper']

# Look at brightness distribution
fig = plt.figure(figsize=(8*len(plt_types),6))

xlims = [ (bins[0], bins[-1]), (0, 50), (bins[-1]-100, bins[-1]) ]
ax_dict = fig.subplot_mosaic([plt_types,])

colors = ['red', 'green', 'blue']
for j, plt_type in enumerate(plt_types):
    
    ax = ax_dict[plt_types[j]]
    for i, color in enumerate(colors):
        
        arr = img[:,:,i] * mm.flight.max_val

        ax.hist(
            arr.flatten(),
            bins = bins,
            color = color,
            histtype = 'step',
        )
        ax.set_xlim(xlims[j])

    ax.set_yscale('log')
    
    ax.set_xlabel( 'Integer Value', )
    ax.set_ylabel( 'Frequency', )

## Inspect a Calibration Image

In [ ]:
image_dir = os.path.join( google_drive_dir, 'Shared drives/NITELite/Data & Analysis/Calibration Tests/3.19.22 Complete Calib Set/23085686_1sec_flatFrames' )

In [ ]:
fp = rng.choice(glob.glob(os.path.join(image_dir, '*.raw')))

In [ ]:
img = mm.flight.get_rgb_img(fp)

In [ ]:
mm.data_viewer.plot_img(img)

In [ ]:
bins = np.arange(-0.5, mm.flight.max_val-0.5, 1)
plt_types = ['big_picture', 'zoomed', 'zoomed_upper']

# Look at brightness distribution
fig = plt.figure(figsize=(8*len(plt_types),6))

xlims = [(bins[0], bins[-1]), (2250, 2500), (3000, 3100)]
ax_dict = fig.subplot_mosaic([plt_types,])

colors = ['red', 'green', 'blue']
for j, plt_type in enumerate(plt_types):
    
    ax = ax_dict[plt_types[j]]
    for i, color in enumerate(colors):
        
        arr = img[:,:,i] * mm.flight.max_val

        ax.hist(
            arr.flatten(),
            bins = bins,
            color = color,
            histtype = 'step',
        )
        ax.set_xlim(xlims[j])

    # ax.set_yscale('log')
    
    ax.set_xlabel( 'Integer Value', )
    ax.set_ylabel( 'Frequency', )

# Georeferencing

In [ ]:
# Get the rotation object
rot = scipy.spatial.transform.Rotation.from_euler(
    'XZY',
    metadata[['imuPitch', 'imuYaw', 'imuRoll']],
    degrees=True,
)

# And now the vector for the center of the nadir camera in wcs
vhat_sensor_center = rot.apply([0, 0, -1])

Given a sensor with position vector $\vec p$ pointed in direction $\hat v$ we want to find the position vector of the source image,

\begin{equation}
\vec s = \vec p + \vec v
\end{equation}

The main unknown in this equation is the magnitude of $\vec v$.
Assuming the sensor is a height $h$ above a flat surface then $h\, /\, \mid \vec v \mid = \cos \phi$, where $\phi$ is the angle between a line connecting the ground and the sensor and $\hat v$.
If $\theta$ is the standard spherical angle determining the z-component of $\hat v$ then $\phi = \pi - \theta$.
Doing some algebra we get $\mid \vec v \mid = - h\,/\, \mid\hat v_z \mid$.
Plugging in and simplifying, we get

\begin{equation}
\vec s = (p_x - h \mid\hat v_x \mid / \mid\hat v_z \mid) \hat x + (p_y - h \mid\hat v_y \mid / \mid\hat v_z \mid) \hat y
\end{equation}

In practice we'll often set $\vec p = \langle 0, 0, h \rangle$ and then deal with the conversion to a uniform reference frame during the conversion to geo-coordinates, simplifying this to 
\begin{equation}
\vec s = \langle - h \mid\hat v_x \mid / \mid\hat v_z \mid,\,\, - h \mid\hat v_y \mid / \mid\hat v_z \mid \rangle
\end{equation}

This is the general solution for any $\hat v$, but the special case we care most about is the center of the image.

In [ ]:
# Distance to center
h = metadata['mAltitude'] - ground_alt

In [ ]:
image_center_coords = - ( h.values / vhat_sensor_center[:,2] )[:,np.newaxis] * vhat_sensor_center[:,:2]

In addition to 

In [ ]:
d_sensor_to_source

In [ ]:
imu_data.loc[imu_data['flight_phase']=='post-flight', 'mAltitude']